### Adaptive Estimator (Single Job Run)
 - Circuit Used - GHZ circuit
 - Qubit count - 21 & 45
 - EMST available - Readout EM, DD, ZNE
 - Circuit Transpilation - Default transpiled with Optimization level = 3

In [2]:
import sys
import os

# Add project root (one level up from 'notebooks')
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [ ]:
# Import requisite libraries
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from adaptive_error_mitigation import adaptive_estimator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
import numpy as np

In [60]:
#Define the GHZ-Echo circuit
NUM_QUBITS = 21

ghz = QuantumCircuit(NUM_QUBITS)
ghz.h(0)
for i in range(NUM_QUBITS - 1):
    ghz.cx(i, i + 1)

In [ ]:
# Define the backend
service = QiskitRuntimeService()
bknd_fez = service.backend("ibm_fez")

In [61]:
# Transpile the circuit
pm = generate_preset_pass_manager(
    optimization_level=3,
    backend=bknd_fez,
    seed_transpiler=42,
)

isa_ghz = pm.run(ghz)

In [62]:
# Create the logical observable (e.g., "ZZZ")
observable = SparsePauliOp("Z" * NUM_QUBITS)

# Apply the layout from the transpiled circuit
isa_obs = observable.apply_layout(isa_ghz.layout)

In [63]:
pub = (isa_ghz, isa_obs)

### Calculating Ideal Expectation value

In [64]:
aer_estimator = AerEstimator()
job = aer_estimator.run([pub])
evs_ideal = job.result()[0].data.evs
print(f"Ideal estimation value of the GHZ Echo crkt: {evs_ideal}")

Ideal estimation value of the GHZ Echo crkt: 0.0


### Running using Qiskit default Estimator

In [65]:
estimator = Estimator(bknd_fez)
job = estimator.run([pub])
print(f"Job-Id: {job.job_id()}")

Job-Id: d5n06659j2ac739lc0e0


In [66]:
job = service.job(job_id="d5n06659j2ac739lc0e0")
evs_def = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using default qiskit estimator: {evs_def}")

Estimation Value of the GHZ Echo crkt using default qiskit estimator: 0.20124405415294547


### Running using Adaptive Estimator

In [74]:
# Run the circuit using adaptive estimator
ae_results = adaptive_estimator.run([pub], backend=bknd_fez)


--- Processing Pub 1/1 ---
--- Initiating Adaptive Error Mitigation and Suppression Framework ---

--> DEFAULT SETTING: Using Default Precision set to 0.015625 and default shots 4096

---> HEURISTIC TRIGGERED: Readout Error Threshold Exceeded
     | Metric: MAX READOUT ERROR - 0.0182 (on Qubit 105)
     | Threshold Set: 0.0100 (READOUT_ERROR_THRESHOLD (config.py))
---> ACTION TAKEN: ENABLED Measure Mitigation (TREX)
     | Resilience Level: 1
     | Measure Noise Learning (Randomizations): 32 (NUM_RANDOMIZATIONS (config.py))
---> ACTION TAKEN: ENABLED Measure Twirling
     | Twirling: Measure=True / Gates=False
     | **Derived Parameters:** shots_per_randomization set to 128 (Shots: 4096 (DEFAULT_SHOTS (config.py)) / Randomizations: 32 (NUM_RANDOMIZATIONS (config.py)))

---> Circuit not scheduled. Applying ALAP scheduling...

---> HEURISTIC TRIGGERED: Circuit density Within Range
     | Metric: CIRCUIT DENSITY (CD) - 0.1081
     | Threshold Range: [0.0700, 0.2500] (config.py)
---> AC

In [76]:
# Retrieving the set estimator options
est_options = ae_results[0]["est_options"]
est_options

max_execution_time,Unset
default_precision,Unset
default_shots,None
resilience_level,2
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [75]:
# Retrieving the job_id
job_id = ae_results[0]["job"].job_id()
print(f"Job-Id: {job_id}")

Job-Id: d5n08rt9j2ac739lc36g


In [77]:
# Retrieve the estimation value
job = service.job(job_id="d5n08rt9j2ac739lc36g")
evs_ae = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using Adaptive Estimator: {evs_ae}")

Estimation Value of the GHZ Echo crkt using Adaptive Estimator: -1.6379472976866876e-17


### Running for 45 qubit GHZ circuit

In [ ]:
# Define the GHZ-Echo circuit
NUM_QUBITS = 45

ghz = QuantumCircuit(NUM_QUBITS)
ghz.h(0)
for i in range(NUM_QUBITS - 1):
    ghz.cx(i, i + 1)

In [87]:
# Transpile the circuit
pm = generate_preset_pass_manager(
    optimization_level=3,
    backend=bknd_fez,
    seed_transpiler=42,
)

isa_ghz = pm.run(ghz)

In [88]:
# Create the logical observable (e.g., "ZZZ")
observable = SparsePauliOp("Z" * NUM_QUBITS)

# Apply the layout from the transpiled circuit
isa_obs = observable.apply_layout(isa_ghz.layout)

In [89]:
pub = (isa_ghz, isa_obs)

### Calculating Ideal Expectation value

In [90]:
aer_estimator = AerEstimator()
job = aer_estimator.run([pub])
evs_ideal = job.result()[0].data.evs
print(f"Ideal estimation value of the GHZ Echo crkt: {evs_ideal}")

Ideal estimation value of the GHZ Echo crkt: 0.0


### Running using Qiskit default Estimator with Resilience level as 2

In [91]:
estimator = Estimator(bknd_fez, options={"resilience_level": 2})
job = estimator.run([pub])
print(f"Job-Id: {job.job_id()}")

Job-Id: d5n0cqc8d8hc73ch5glg


In [103]:
estimator = Estimator(bknd_fez, options={"resilience_level": 2})
estimator.options

max_execution_time,Unset
default_precision,Unset
default_shots,Unset
resilience_level,2
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [92]:
job = service.job(job_id="d5n0cqc8d8hc73ch5glg")
evs_def = job.result()[0].data.evs
print(
    f"Estimation Value of the GHZ Echo crkt using default qiskit estimator with resilience level as 2: {evs_def}"
)

Estimation Value of the GHZ Echo crkt using default qiskit estimator with resilience level as 2: -0.052744671057252265


### Running using Adaptive Estimator

In [97]:
# Run the circuit using adaptive estimator
ae_results = adaptive_estimator.run([pub], backend=bknd_fez)


--- Processing Pub 1/1 ---
--- Initiating Adaptive Error Mitigation and Suppression Framework ---

--> DEFAULT SETTING: Using Default Precision set to 0.015625 and default shots 4096

---> HEURISTIC TRIGGERED: Readout Error Threshold Exceeded
     | Metric: MAX READOUT ERROR - 0.0922 (on Qubit 2)
     | Threshold Set: 0.0100 (READOUT_ERROR_THRESHOLD (config.py))
---> ACTION TAKEN: ENABLED Measure Mitigation (TREX)
     | Resilience Level: 1
     | Measure Noise Learning (Randomizations): 32 (NUM_RANDOMIZATIONS (config.py))
---> ACTION TAKEN: ENABLED Measure Twirling
     | Twirling: Measure=True / Gates=False
     | **Derived Parameters:** shots_per_randomization set to 128 (Shots: 4096 (DEFAULT_SHOTS (config.py)) / Randomizations: 32 (NUM_RANDOMIZATIONS (config.py)))

---> Circuit not scheduled. Applying ALAP scheduling...

Final Estimator Options:
{'_VERSION': 2, 'max_execution_time': Unset, 'environment': EnvironmentOptions(log_level='WARNING', callback=None, job_tags=None, private

In [98]:
# Retrieving the set estimator options
est_options = ae_results[0]["est_options"]
est_options

max_execution_time,Unset
default_precision,Unset
default_shots,None
resilience_level,1
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [99]:
# Retrieving the job_id
job_id = ae_results[0]["job"].job_id()
print(f"Job-Id: {job_id}")

Job-Id: d5n0es48d8hc73ch5ir0


In [100]:
# Retrieve the estimation value
job = service.job(job_id="d5n0es48d8hc73ch5ir0")
evs_ae = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using Adaptive Estimator: {evs_ae}")

Estimation Value of the GHZ Echo crkt using Adaptive Estimator: 0.32684824902723736


### Conclusion
 - The estimation value improved by circa 35% while using the Adaptive Estimator instead of using the default Qiskit estimator